In [9]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from pathlib import Path
import numpy as np
import math
from scipy.stats import levene, ttest_ind
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV
from sklearn.utils import shuffle
from scipy.stats import pearsonr

from utils import get_target_group
from utils import save_to_pickle
from utils import load_pickle

In [2]:
p = Path('./resources/temp_file/final_extract_results.pkl')
final_results_df = pd.read_pickle(p)

In [3]:
final_results_df: pd.DataFrame
final_results_df.iloc[[0, 1]]

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,gradient_glszm_ZoneVariance,gradient_ngtdm_Busyness,gradient_ngtdm_Coarseness,gradient_ngtdm_Complexity,gradient_ngtdm_Contrast,gradient_ngtdm_Strength,label,process_index,stem,patient_ID
0,v3.1.0,1.21.6,2.2.1,1.3.0,3.7.9,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'Wavelet': {}, 'LoG': {}, 'Lo...",ed56dfd16b94093bb78ace3a0bd739f8e0d877c2,3D,"(1.0, 1.0, 1.0)",...,2.175838e+06,10.218170,0.049777,0.008120,0.000045,0.050537,1,0,B-00073B8A5734D5E5D21C6E9B04299C3F,00001E
1,v3.1.0,1.21.6,2.2.1,1.3.0,3.7.9,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'Wavelet': {}, 'LoG': {}, 'Lo...",8ef55464a40ae22f61f193f0992c28e56d1cf5ef,3D,"(1.0, 1.0, 1.0)",...,1.533044e+07,321.873332,0.000793,0.122848,0.000212,0.003918,1,1,B-000C017A869153D099C1AA9B0C892CF1,00002E


In [4]:
# 数据集的划分
p_train = Path('./resources/all_images/Groups/train_ids.csv')
train_df = pd.read_csv(p_train)
train_label = train_df['ids']
train_data = get_target_group(train_label, final_results_df).iloc[:, 22:-3]

p_val = Path('./resources/all_images/Groups/validation_ids.csv')
val_df = pd.read_csv(p_val)
val_label = val_df['ids']
val_data = get_target_group(val_label, final_results_df).iloc[:, 22:-3]

p_test = Path('./resources/all_images/Groups/test_ids.csv')
test_df = pd.read_csv(p_test)
test_label = test_df['ids']
test_data = get_target_group(test_label, final_results_df).iloc[:, 22:-3]

In [5]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(13772, 1131)
(4593, 1131)
(4691, 1131)


In [6]:
# t检验
# 数据分组，数据按 label=0 和 label=1 分组
train_data_0 = train_data[train_data['label'] == 0]
train_data_1 = train_data[train_data['label'] == 1]
print(train_data_0.shape, train_data_1.shape)

(4944, 1131) (8828, 1131)


In [7]:
# t检验特征筛选
t_test_index = []
for colName in train_data.columns[:]:
    if levene(train_data_0[colName], train_data_1[colName])[1] > 0.05:
        if ttest_ind(train_data_0[colName], train_data_1[colName])[1] < 0.05:
            t_test_index.append(colName)
    else:
        if ttest_ind(train_data_0[colName], train_data_1[colName], equal_var=False)[1] < 0.05:
            t_test_index.append(colName)
print(len(t_test_index), t_test_index[0], t_test_index[-1])

D:\exe_code\Python\17_Echinococcosis_ML\venv\lib\site-packages\scipy\stats\morestats.py:2603: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom


1062 original_shape_Elongation label


In [8]:
# t检验后 训练集 数据整理
train_data_0 = train_data_0[t_test_index]
train_data_1 = train_data_1[t_test_index]
train_data = pd.concat([train_data_0, train_data_1])

X_train = train_data[train_data.columns[:-1]]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = t_test_index[:-1]

y_train = train_data['label']
print(X_train.shape, y_train.shape)

(13772, 1061) (13772,)


In [9]:
# t检验后 验证集 数据整理
val_data = val_data[t_test_index]

X_val = val_data[val_data.columns[:-1]]

X_val = scaler.transform(X_val)
X_val = pd.DataFrame(X_val)
X_val.columns = t_test_index[:-1]

y_val = val_data['label']
print(X_val.shape, y_val.shape)

(4593, 1061) (4593,)


In [10]:
# t检验后 测试集 数据整理
test_data = test_data[t_test_index]

X_test = test_data[test_data.columns[:-1]]

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test)
X_test.columns = t_test_index[:-1]

y_test = test_data['label']
print(X_test.shape, y_test.shape)

(4691, 1061) (4691,)


In [11]:
# Pearson相关系数
pearson_index = []
statistic_list = []
for colName in X_train.columns[:]:
    statistic, p_value = pearsonr(X_train[colName], y_train)
    statistic_list.append(statistic)
    if p_value < 0.05:
        pearson_index.append(colName)
print(len(pearson_index))

1059


In [12]:
# Pearson相关系数 数据整理
X_train = X_train[pearson_index]
X_val = X_val[pearson_index]
X_test = X_test[pearson_index]
print(X_train.shape, X_val.shape, X_test.shape)

(13772, 1059) (4593, 1059) (4691, 1059)


In [ ]:
# Lasso回归
alphas = np.logspace(-4, 1, 100)  # alpha就是损失函数的lambda
model_lassoCV = LassoCV(alphas=alphas, max_iter=2000, cv=5).fit(X_train, y_train)

In [16]:
# 临时保存下
save_to_pickle(Path('./resources/temp_file/X_train.pkl'), X_train)
save_to_pickle(Path('./resources/temp_file/y_train.pkl'), y_train)
save_to_pickle(Path('./resources/temp_file/X_val.pkl'), X_val)
save_to_pickle(Path('./resources/temp_file/y_val.pkl'), y_val)
save_to_pickle(Path('./resources/temp_file/X_test.pkl'), X_test)
save_to_pickle(Path('./resources/temp_file/y_test.pkl'), y_test)
save_to_pickle(Path('./resources/temp_file/model_lassoCV.pkl'), model_lassoCV)

In [2]:
# 加载数据
X_train = load_pickle(Path('./resources/temp_file/X_train.pkl'))
y_train = load_pickle(Path('./resources/temp_file/y_train.pkl'))
X_val = load_pickle(Path('./resources/temp_file/X_val.pkl'))
y_val = load_pickle(Path('./resources/temp_file/y_val.pkl'))
X_test = load_pickle(Path('./resources/temp_file/X_test.pkl'))
y_test = load_pickle(Path('./resources/temp_file/y_test.pkl'))
model_lassoCV = load_pickle(Path('./resources/temp_file/model_lassoCV.pkl'))

In [4]:
# coef_代表LASSO计算出来的每个特征的系数
coef = pd.Series(model_lassoCV.coef_, index=X_train.columns)
lasso_index = coef[coef != 0].index
print('Lasso选择出的系数不为0的特征的个数:', len(lasso_index), lasso_index[0], lasso_index[-1])

Lasso选择出的系数不为0的特征的个数: 679 original_shape_Elongation gradient_ngtdm_Strength


In [5]:
# lasso回归后数据处理
X_train = X_train[lasso_index]
X_val = X_val[lasso_index]
X_test = X_test[lasso_index]

In [6]:
print(X_train.shape, X_val.shape, X_test.shape)

(13772, 679) (4593, 679) (4691, 679)


In [12]:
# 递归特征消除
estimator = LogisticRegressionCV(class_weight='balanced', cv=5, max_iter=1000, solver='sag')
rfe = RFE(estimator=estimator, n_features_to_select=30, step=1)
rfe = rfe.fit(X_train, y_train)

/Users/kukudehui/Documents/exec_code/python/17_Echinococcosis_ML/venv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/kukudehui/Documents/exec_code/python/17_Echinococcosis_ML/venv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/kukudehui/Documents/exec_code/python/17_Echinococcosis_ML/venv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/kukudehui/Documents/exec_code/python/17_Echinococcosis_ML/venv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/kukudehui/Documents/exec_code/python/17_E

KeyboardInterrupt: 